In [ ]:
#Imports
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#Comment this lines if you have this stuff already installed
#!(yes |pip install geopandas)
#!(yes |pip install descartes)
#!(yes |conda install -c conda-forge geoplot)
#!(yes | pip install plotly)
import plotly.express as px
#!pip install spacy
#!python -m spacy download en_core_web_sm
#nltk.download('wordnet')
#nltk.download('punkt')
import nltk
from nltk.corpus import stopwords
from nltk.tag import pos_tag
import spacy
import en_core_web_sm
#!pip install gensim
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import KeyedVectors
from gensim import models
import gensim

#!pip install catboost
!pip install xgboost
import xgboost as xgb
from catboost import CatBoostRegressor

%config IPCompleter.greedy=True
%matplotlib inline

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs
from sklearn.decomposition import PCA
from matplotlib import pyplot
import warnings
warnings.filterwarnings('ignore')
from numpy import linalg as LA
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.integrate import odeint
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn import tree
from gensim import models
from gensim.models import Word2Vec


In [ ]:
#!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
#file = 'GoogleNews-vectors-negative300.bin.gz'

#model = KeyedVectors.load_word2vec_format (file, binary=True)
#model.wv.similar_by_word('mutation')
#if you never ran the above code, then comment the code below and descomment the code above
model = KeyedVectors.load('model.bin')

In [ ]:
#Gets the hashtag or user
def getter(text, char):
    pos = text.find(char)
    text = text[pos:]
    #Some users or hashtags finish with : or .
    if text.endswith(':') or text.endswith('.'):
        text = text[:-1]
    return text
def validUser(userName):
    if '@' in userName:
        user = getter(userName, '@')
        length = len(user)
        if (length > 1 and length <= 16):
            for char in user[1:]:
                if not(char.isalnum() or char == '_'): return False
            return True
    return False
def validLink(link):
    type1 = 'https://'
    type2 = 'http://'
    if type1 in link and len(link) > 9: return True
    if type2 in link and len(link) > 8: return True
    return False
def validHashtag(hashtag):
    if '#' in hashtag:
        hashtag = getter(hashtag, '#')
        hashtag = hashtag[1:]
        return hashtag.isalnum()
    return False

In [ ]:
tweetsInfo = pd.read_csv('./ToChangeKeywordsAndLocations/train.csv')
def analyzeTweets(text):
    result = [0,0,0] #number of usersTagged, hashtags and links
    text = text.split()
    for word in text:
        if validUser(word): result[0] += 1
        elif validHashtag(word): result[1] += 1
        elif validLink(word): result[2] += 1
    return result
def getInfo(dataList, pos):
    return dataList[pos]
#Aux column to get the result
tweetsInfo['aux_column'] = tweetsInfo.text.apply(analyzeTweets)

tweetsInfo['users_tagged'] = tweetsInfo.aux_column.apply(getInfo,args=(0,))
tweetsInfo['hashtags'] = tweetsInfo.aux_column.apply(getInfo,args=(1,))
tweetsInfo['links'] = tweetsInfo.aux_column.apply(getInfo,args=(2,))

del tweetsInfo['aux_column']
tweetsInfo.head()

In [ ]:
def sizes(size):
    if size == 'small': return 0
    if size == 'medium': return 5
    return 10
def tweetSize(tweetLength, minValue, intervalRange):
    if tweetLength < (minValue + intervalRange): return 'small'
    if (minValue + intervalRange) <= tweetLength and tweetLength < (minValue + 2 * intervalRange): return 'medium'
    return 'large'

tweetsInfo['tweet_length'] = tweetsInfo.text.str.len()
#Parallel coordinates to show the characteristics of the tweets
#First we get the range of each interval
minValue = tweetsInfo.tweet_length.min()
maxValue = tweetsInfo.tweet_length.max()
intervalRange = (maxValue - minValue) // 3

#We add a new column
tweetsInfo['tweet_size'] = tweetsInfo.tweet_length.apply(tweetSize, args = (minValue, intervalRange))

disastersDf = tweetsInfo
disastersDf.head()


In [ ]:
world = pd.read_csv("./ToChangeKeywordsAndLocations/worldcities.csv", usecols = ['city_ascii','iso3' , 'country'])
worldIsoSet = set(world.iso3)
nlp = spacy.load("en_core_web_sm")
def findInDataset(locationText):
    location = re.sub(",|\.", "", locationText)
    for possibleCountry in location.split():
        if possibleCountry in  worldIsoSet: return 1
    return 0
def hasValidLocation(locationText):
    if pd.isnull(locationText): return 0
    doc = nlp(locationText)
    for ent in doc.ents:
        if ent.label_ == 'GPE':
            return 1
    return findInDataset(locationText)
disastersDf['Valid_location'] = disastersDf.location.apply(hasValidLocation)
disastersDf['tweet_size'] = disastersDf.tweet_size.apply(sizes)
disastersDf.head()

In [ ]:
X, y = disastersDf.iloc[:,5:], disastersDf.iloc[:,4]  #X tiene que tener todos los features distintos al target

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

In [ ]:
rf_model = RandomForestClassifier(random_state=1)
rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

In [ ]:
trueTweets = disastersDf[disastersDf.target == 1]
falseTweets = disastersDf[disastersDf.target == 0]
falseTexts = falseTweets.text.to_list()
trueTexts = trueTweets.text.to_list()
falseSentences = [text.split() for text in falseTexts]
trueSentences = [text.split() for text in trueTexts]
falseModel = Word2Vec(falseSentences, min_count = 25)
trueModel = Word2Vec(trueSentences, min_count = 25)
print(falseModel)
words = list(falseModel.wv.vocab)
print(words)

In [ ]:
disastersDf['Total_elements'] = disastersDf.hashtags + disastersDf.users_tagged + disastersDf.links
disastersDf['links_hash'] = disastersDf.hashtags + disastersDf.links
disastersDf['links_users'] = disastersDf.users_tagged + disastersDf.links
disastersDf['hash_users'] = disastersDf.hashtags + disastersDf.users_tagged
train = disastersDf.iloc[:, 4:]
train

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
def get_average_vector(text):
    tokens = [w.lower() for w in word_tokenize(text) if w.lower() not in stop_words]
    return np.mean(np.array([model.wv[w] for w in tokens if w in model]), axis=0)

model.wv.similar_by_vector(get_average_vector(disastersDf.loc[0, 'text']))

In [ ]:
vector_list = [model[word] for word in words if word in model.vocab]

# Create a list of the words corresponding to these vectors
words_filtered = [word for word in words if word in model.vocab]

# Zip the words together with their vector representations
word_vec_zip = zip(words_filtered, vector_list)

# Cast to a dict so we can turn it into a DataFrame
word_vec_dict = dict(word_vec_zip)
df = pd.DataFrame.from_dict(word_vec_dict, orient='index')

In [ ]:
model.get_vector('fill').sum()

In [ ]:
from nltk.corpus import wordnet
class RepeatReplacer(object):
    def __init__(self):
        self.repeat_regexp = re.compile(r'(\w*)(\w)\2(\w*)')
        self.repl = r'\1\2\3'
    def replace(self, word):
        if wordnet.synsets(word):
            return word
        repl_word = self.repeat_regexp.sub(self.repl, word)
        
        if repl_word != word:
            return self.replace(repl_word)

        return repl_word

In [ ]:
import re
replacement_patterns = [
(r'won\'t', 'will not'),
(r'can\'t', 'cannot'),
(r'i\'m', 'i am'),
(r'I\'m', 'I am'),
(r'ain\'t', 'is not'),
(r'(\w+)\'ll', '\g<1> will'),
(r'(\w+)n\'t', '\g<1> not'),
(r'(\w+)\'ve', '\g<1> have'),
(r'(\w+)\'s', '\g<1> is'),
(r'(\w+)\'re', '\g<1> are'),
(r'(\w+)\'d', '\g<1> would')
]
class RegexpReplacer(object):
    def __init__(self, patterns=replacement_patterns):
        self.patterns = [(re.compile(regex), repl) for (regex, repl) in patterns]
    def replace(self, text):
        s = text
        for (pattern, repl) in self.patterns:
            (s, count) = re.subn(pattern, repl, s)
        return s

In [ ]:
def cleanTweet(tweet): #Modificado para sacar solo los links
    return ' '.join(re.sub("(\w+:\/\/\S+)", " ", tweet).split())
import string
def deletePunctuation(tokenizedText):
    x = re.compile('[%s]' % re.escape(string.punctuation))
    tokenized_text_no_punctuation = []
    for token in tokenizedText: #Agarro las palabras de la lista
        newToken = x.sub(u'', token)
        if not newToken == u'':
            tokenized_text_no_punctuation.append(newToken)
    return tokenized_text_no_punctuation
def deleteStopwords(tokenizedText, stopwords):
    return [word for word in tokenizedText if word not in stopwords]
def editText(text, stopwords, replacer, repeatReplacer):
    #Primero elimino los links
    text = cleanTweet(text)
    
    #Paso a lower el text
    text = text.lower()
    
    #Reemplazo los I'm por I am
    text = replacer.replace(text)
    
    #Elimino los caracteres repetidos, ej: ohhh por oh
    words = text.split()
    text = ' '.join(repeatReplacer.replace(word) for word in words)
    
    #Tokenizo el texto
    tokenizedText = nltk.word_tokenize(text)
    
    #Elimno los signos de puntuacion
    tokenizedText = deletePunctuation(tokenizedText)
    
    #Elimino los stopwords
    tokenizedText = deleteStopwords(tokenizedText, stopwords)
    
    editText = ' '.join(tokenizedText)
    return editText


In [ ]:
repeatReplacer = RepeatReplacer()
replacer = RegexpReplacer()
stop = set(stopwords.words('english'))

disastersDf['text'] = disastersDf.text.apply(editText, args = (stop, replacer, repeatReplacer))
disastersDf.head()

In [ ]:
def getOp(value, operation, vectorOp = False):
    if vectorOp:
        finalValue = 0
        for word in value.split():
            if word in model.vocab:
                finalValue = operation(model.get_vector(word), finalValue)
        return finalValue
        
    finalValue = [operation(model.get_vector(i)) for i in value.split() if i in model.vocab]
    return finalValue if len(finalValue) and not np.isnan(finalValue).any() else 0

disastersDf['textProm'] = disastersDf.text.apply(lambda x: np.mean(getOp(x, sum)))
disastersDf['textMeanStd'] = disastersDf.text.apply(lambda x: np.mean(getOp(x, np.std)))
disastersDf['textStd'] = disastersDf.text.apply(lambda x: np.std(getOp(x, np.add, True)))
disastersDf['textMean'] = disastersDf.text.apply(lambda x: np.mean(getOp(x, np.add, True)))
disastersDf['textMeanMean'] = disastersDf.text.apply(lambda x: np.mean(getOp(x, np.mean)))


In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")
def forApply(row):
    words = nlp(row.text)
    stopTrue = 0
    alphaTrue = 0
    verbs = 0
    for token in words:
        stopTrue += 1 if token.is_stop else 0
        alphaTrue += 1 if token.is_alpha else 0
        verbs += 1 if token.tag_.startswith('VB') else 0
    row['verbs'], row['stopTrue'], row['alphaTrue'] = verbs, stopTrue, alphaTrue
    return row
disastersDf = disastersDf.apply(forApply, axis = 1, result_type = 'expand')

In [ ]:
dataPage = pd.read_html('https://en.wikipedia.org/wiki/List_of_most-followed_Twitter_accounts')
top50Users = {username.lower()[1:] for username in dataPage[0].iloc[:,2]}
def hasTopUser(text):
    words = text.split()
    for word in words:
        if word in top50Users: return 1
    return 0
disastersDf['hasTopUser'] = disastersDf.text.apply(hasTopUser)

In [ ]:
originalTweets = pd.read_csv('./train.csv', usecols = ['text'])
disastersDf['length_proportion'] = disastersDf.text.str.len() / disastersDf.tweet_length

disastersDf['amount_of_words_proportion'] = disastersDf.text.apply(lambda x: len(x.split())) / originalTweets.text.apply(lambda x: len(x.split()))

In [ ]:
pornwords = {'sex','sexy', 'cunt', 'dick', 'cock', 'xxx', 'porn',\
                 'lesbian', 'gay', 'masturbation', 'fap', 'asshole',\
                 'assholes', 'suck', 'sucker', 'idiot', 'stupid', 'cum',\
                 'blowjob', 'bitch', 'slut', 'sluts', 'whores', 'bitches', 'whore',\
                 'cunts', 'suckers', 'ass', 'butt', 'nude', 'nudes', 'naked', 'fucking',\
                 'xoxo', 'cocks', 'dicks', 'wtf', 'lol', 'lmfao', 'lmao', 'cunts', 'jerkface'}

def pornWords(sentence): #VER QUE MIERDA HACER CON PALABRAS COMO LOL, WTF, ETC
    value = 0
    for word in sentence.split():
        if word.lower() in pornwords:
            value += 1
    
    return value

disastersDf['Porn_words'] = disastersDf.text.apply(pornWords)


In [ ]:
def selectColumns(dataFrame, posTarget): #X tiene que tener todos los features distintos al target
    X, y = dataFrame.iloc[:, posTarget + 1:], dataFrame.iloc[:, posTarget]
    return X, y

X, y = selectColumns(disastersDf, 4)

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier

def predictionPercentage(X_train, y_train, X_test, y_test, graphic = True):
    predictions = {}
    
    #Random forest
    rf_model = RandomForestClassifier(criterion= "entropy",
		max_depth= 30,
		min_samples_leaf= 5,
		min_samples_split= 5,
		n_estimators= 1200)
    rf_model.fit(X_train, y_train)
    preds = rf_model.predict(X_test)
    predictions['RandomForest'] = accuracy_score(y_test, preds.round())
    #np.sqrt(mean_squared_error(y_test, preds.round()))
    if graphic:
        plt.bar(X_train.columns, rf_model.feature_importances_)
        plt.xlabel('Features')
        plt.ylabel('Importancia')
        plt.title('Importancia Features con RandomForest')
        plt.xticks(rotation = 90, horizontalalignment = 'right')
        plt.show()
    
    #Naive bayes
    gnb = GaussianNB()
    y_pred = gnb.fit(X_train, y_train).predict(X_test)
    predictions['NaiveBayes'] = accuracy_score(y_test, y_pred.round())
    #np.sqrt(mean_squared_error(y_test, y_pred.round()))
    
    #Tree
    clf = tree.DecisionTreeClassifier(min_samples_split = 10)
    y_pred = clf.fit(X_train, y_train).predict(X_test)
    predictions['Tree'] = accuracy_score(y_test, y_pred.round())
    #np.sqrt(mean_squared_error(y_test, y_pred.round()))
    
    #XGBoost
    xg_reg = xgb.XGBClassifier(colsample_bytree= 0.8,
		gamma= 1,
		max_depth = 3,
		min_child_weight = 5,
		subsample = 1.0)

    xg_reg.fit(X_train,y_train)

    preds = xg_reg.predict(X_test)
    predictions['XGBoost'] = accuracy_score(y_test, preds.round())
    #np.sqrt(mean_squared_error(y_test, preds.round()))
    
#    clf = GradientBoostingClassifier(random_state=0)
#    clf.fit(X_train, y_train)
#    y_pred = clf.predict(X_test[:2])
 #   predictions['GBC'] = accuracy_score(y_test, y_pred.round())
    
    return predictions

def printPredictions(dicc):
    #print('Predicciones (1 - MSE)\n')
    for key, value in dicc.items():
        print('{}: {}\n'.format(key, value))   

pred = predictionPercentage(X_train, y_train, X_test, y_test)

printPredictions(pred)

    

In [ ]:
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.4, train_size = 0.6, random_state=123) #Cambio los parametros

printPredictions(predictionPercentage(X_train, y_train, X_test, y_test))

In [ ]:
#A estas alturas el texto ya esta tocado
def standarNorm(dataFrame, labels):
    normMethod = '_standardNorm'
    for label in labels:
        mean = dataFrame[label].mean()
        std = dataFrame[label].std()
        dataFrame[label + normMethod] = dataFrame[label].apply(lambda x: (x - mean) / std)

def minMaxNorm(dataFrame, labels):
    normMethod = '_minMaxNorm'
    for label in labels:
        minValue = dataFrame[label].min()
        maxValue = dataFrame[label].max()
        dataFrame[label + normMethod] = dataFrame[label].apply(lambda x: (x - minValue) / (maxValue - minValue))

def tweetElementsNorm(dataFrame, labels):
    labels = ['hashtags', 'users_tagged', 'links', 'Porn_words',\
             'links_hash', 'links_users', 'hash_users']
    normMethod = '_natNat'
    dataFrame['auxCol'] = dataFrame.text.apply(lambda x: len(x.split()))
    for label in labels:
        if label == 'links': #Caso borde porque volamos los links del text
            dataFrame[label + normMethod] = dataFrame[label] / (dataFrame.auxCol + dataFrame.links)
            continue            
            
        dataFrame[label + normMethod] = dataFrame[label] / dataFrame.auxCol
    del dataFrame['auxCol']


def applyNormalizations(dataFrame):
    substract = {'textProm', 'textMeanStd', 'textStd', 'textMean', 'norm2', 'Valid_location'}
    normalization = [standarNorm, minMaxNorm, tweetElementsNorm]
    labels = list(set(disastersDf.iloc[1:2, 5:].columns) - substract)
    for normOp in normalization:
        normOp(dataFrame, labels)

applyNormalizations(disastersDf)
disastersDf.head()

rowsToNormalizeSimple = ['users_tagged', 'hashtags', 'Porn_words']
def getColsChanged(row):
    amountOfWords = len(row['text'].split())
    for element in rowsToNormalizeSimple:
        row[element] = row[element] / amountOfWords
    row['links'] = row['links'] / (row['links'] + amountOfWords)
    return row
disastersDf = disastersDf.apply(getColsChanged, axis = 1)
disastersDf.head()

In [ ]:
#disastersDf[disastersDf['Porn_words'] != 0.00].Porn_words.value_counts()
#El 1.0 esta chequeado, el tweet dice: "@Hurricane_Dame ???????? I don't have them they out here"

disastersDf['links_hash'] = disastersDf.links + disastersDf.hashtags
disastersDf['links_users'] = disastersDf.links + disastersDf.users_tagged
disastersDf['hash_users'] = disastersDf.users_tagged + disastersDf.hashtags

#Normalizo las longitudes
meanLength = disastersDf.tweet_length.mean()
stdLength = disastersDf.tweet_length.std()
disastersDf['tweet_length_normalized'] = (disastersDf.tweet_length - meanLength) / stdLength

In [ ]:
def norm2(text):
    norm = 0
    for word in text.split():
        if word in model.vocab:
            norm += np.linalg.norm(model.get_vector(word))
    return norm

disastersDf['norm2'] = disastersDf.text.apply(norm2)

In [ ]:
disastersDf.columns

In [ ]:
train = disastersDf[['target', 'tweet_length', 'hashtags', 'users_tagged',\
                        'links_hash', 'links_users', 'norm2', 'textStd', 'textProm']]
#[['target', 'tweet_length', 'hashtags', 'users_tagged', 'Porn_words', 'Valid_location','links_hash', 'links_users', 'hash_users', 'norm2', 'prom', 'std']]
X, y = selectColumns(train, 0)

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.1, random_state=41)

printPredictions(predictionPercentage(X_train, y_train, X_test, y_test))

In [ ]:
#posMax = 10
#maxValue = -1
#for i in range(1,100):
#    X_train, X_test, y_train, y_test = \
#    train_test_split(X, y, test_size=i/100, random_state=41)
#    actualValue = predictionPercentage(X_train, y_train, X_test, y_test)['XGBoost']
#    if actualValue > maxValue:
#        posMax = i/100
#        maxValue = actualValue

In [ ]:
X, y = selectColumns(disastersDf, 4)

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.1, random_state=41)

printPredictions(predictionPercentage(X_train, y_train, X_test, y_test))

In [ ]:
disastersDf.columns

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
text = [disastersDf.loc[0,'text']]
voca = vectorizer.fit(disastersDf.text.values)

In [ ]:
#from sklearn.naive_bayes import CategoricalNB  NO FUNCIONA CON VALORES NEGATIVOS
#clf = CategoricalNB()
#clf.fit(X_train, y_train)
#y_pred = clf.predict(X_test)
#accuracy_score(y_test, y_pred.round())



#loss="hinge": (soft-margin) linear Support Vector Machine,
#loss="modified_huber": smoothed hinge loss,
#loss="log": logistic regression,
#penalty="l2": L2 norm penalty on coef_.
#penalty="l1": L1 norm penalty on coef_.
#penalty="elasticnet": Convex combination of L2 and L1; (1 - l1_ratio) * L2 + l1_ratio * L1.

#from sklearn.linear_model import SGDClassifier  #FUNCIONA PERO DA distinto cada vez que corro
#clf = SGDClassifier(loss="log", penalty="l2", max_iter=5)
#clf.fit(X_train, y_train)
#y_pred = clf.predict(X_test)
#accuracy_score(y_test, y_pred.round())

#from sklearn.ensemble import ExtraTreesClassifier #No lo probe

from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                     hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(X_train, y_train)

clf2 = MLPClassifier(alpha=1e-05, hidden_layer_sizes=(5, 2), random_state=3,
              solver='lbfgs')

clf2.fit(X_train, y_train)

y_pred = clf.predict(X_test)
accu1 = accuracy_score(y_test, y_pred.round())

y_pred = clf2.predict(X_test)
accu2 = accuracy_score(y_test, y_pred.round())

clf3 = MLPClassifier(solver='lbfgs', alpha=1e-5,
                     hidden_layer_sizes=(15,), random_state=5)  #Esta funciona mejor que las otras

clf3.fit(X_train, y_train)
y_pred = clf3.predict(X_test)
accu3 = accuracy_score(y_test, y_pred.round())


accu1, accu2, accu3,

In [ ]:
from sklearn.linear_model import SGDClassifier  #FUNCIONA PERO DA distinto cada vez que corro
clf = SGDClassifier(loss="log", penalty="l2", max_iter=5)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred.round())

In [ ]:
disastersDf

In [ ]:
disastersDf.columns

In [ ]:
!pip install mlxtend
from mlxtend.feature_selection import ExhaustiveFeatureSelector
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import roc_auc_score

feature_selector = ExhaustiveFeatureSelector(RandomForestClassifier(n_jobs=-1),
           min_features=2,
           max_features=4,
           scoring='roc_auc',
           print_progress=True,
           cv=2)

## NLTK

In [ ]:
world = pd.read_csv("./ToChangeKeywordsAndLocations/worldcities.csv", usecols = ['city_ascii','iso3' , 'country'])
worldIsoSet = set(world.iso3)

In [ ]:
worldIsoSet = set(world.iso3)

In [ ]:

doc = nlp("Birmingham")

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

In [ ]:
disastersDf.location

In [ ]:
#coso.value_counts()
disastersDf.location.value_counts()

In [ ]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
from gensim import models
from sklearn.decomposition import PCA
from matplotlib import pyplot


trueTweets = disastersDf[disastersDf.target == 1]
falseTweets = disastersDf[disastersDf.target == 0]

falseTexts = falseTweets.text.to_list()
trueTexts = trueTweets.text.to_list()
falseSentences = [text.split() for text in falseTexts]
trueSentences = [text.split() for text in trueTexts]
falseModel = Word2Vec(falseSentences, min_count = 1)
trueModel = Word2Vec(trueSentences, min_count = 1)

X = trueModel[trueModel.wv.vocab]
Y = falseModel[falseModel.wv.vocab]
pca = PCA(n_components=2)
result = pca.fit_transform(X)
result2 = pca.fit_transform(Y)
# create a scatter plot of the projection
pyplot.scatter(result[:, 0], result[:, 1], c = 'g')
pyplot.scatter(result2[:, 0], result2[:, 1], c = 'r');

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X_train, y_train)
alternativeModel = SelectFromModel(lsvc, prefit=True)
X_new = alternativeModel.transform(X_train)
X_new

In [ ]:
newDataFrame = pd.read_csv('./train.csv')
def addToHash(hashToUse, row):
    sentence = row['text']
    isTrue = row['target']
    for word in sentence.lower().split():
        hashToUse[word] = hashToUse.get(word, np.array([0, 0])) + [isTrue, 1]
    return row
amountOfWords = {}
newDataFrame['truthProb'] = newDataFrame.apply(lambda x: addToHash(amountOfWords, x), axis = 1)\
.text.apply(lambda x: np.mean([amountOfWords[i][0]  for i in x.lower().split()]))


In [ ]:
len([0 for i in range(300)])

In [ ]:
train = disastersDf[['text', 'target', 'tweet_length', 'alphaTrue', 'textStd', 'textProm']]
#[['target', 'tweet_length', 'hashtags', 'users_tagged', 'Porn_words', 'Valid_location','links_hash', 'links_users', 'hash_users', 'norm2', 'prom', 'std']]
#[['target', 'tweet_length', 'users_tagged',\
  #                       'norm2', 'textStd', 'textProm']]

train = train.join(train.apply(lambda x: model.get_vector(x['text'].split()[0]) if x['text'].split()[0] in model.vocab else [0 for i in range(300)], axis = 1, result_type = 'expand'))
X, y = selectColumns(train, 1)

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.1, random_state=41)

printPredictions(predictionPercentage(X_train, y_train, X_test, y_test, False))

In [ ]:
train = disastersDf[['text', 'target', 'tweet_length', 'alphaTrue', 'textStd', 'textProm']]
#[['target', 'tweet_length', 'hashtags', 'users_tagged', 'Porn_words', 'Valid_location','links_hash', 'links_users', 'hash_users', 'norm2', 'prom', 'std']]
#[['target', 'tweet_length', 'users_tagged',\
  #                       'norm2', 'textStd', 'textProm']]

train = train.join(train.apply(lambda x: model.get_vector(x['text'].split()[0]) if x['text'].split()[0] in model.vocab else [0 for i in range(300)], axis = 1, result_type = 'expand'))
train

In [ ]:
#For hiperParameters
#disastersDf.to_csv('forHiper', index = False)